In [223]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from datetime import timedelta
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)

# read the csv into a dataframe
# only import the columns we will use, so avoid spending time dropping 90 columns later
#df = pd.read_csv(('0714monm135536.csv'), usecols=['propertyLocation', 'ownersName', 'ownersMailingAddress', 'cityStateZip'])
df = pd.read_pickle('newark.pkl')
# after this initial instance, we'll read from the pickled dataframes
errors_df = pd.read_pickle('errors_newark.pkl')

# create a column for the owner's full mailing address
#df['ownersFullMailingAddress'] = df['ownersMailingAddress'] + ', ' + df.cityStateZip
# create a column for the property's full address
#df['propertyFullAddress'] = df.propertyLocation + ' Newark, NJ'

#df = pd.read_pickle('newark.pkl')
#errors_df = pd.read_pickle('errors_df.pkl')

# returns an array of properties associated with the owne
def getPropertiesOwned(owner=str):
    unique_properties = df[df.ownersName == owner]['propertyLocation'].unique()    
    return unique_properties.tolist()

# number of times property appears in first column
# I'm assuming this is one way to determine how many units
# are at a given address
def getUnits(address=str):
    return len(df[df.propertyLocation == address])

# number of the properties owned by the given owner
def getNumberOwned(owner=str):
    return df[(df.ownersName == owner)]['propertyLocation'].count()

In [224]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
0,1050-1052 MCCARTER HWY,SEABRA BROTHERS II,260CHESTNUT ST. 2ND FLOOR,"NEWARK, NJ 07105",1,[1050-1052 MCCARTER HWY],1,"1050-1052 MCCARTER HWY Newark, NJ","(1050, McCarter Highway, Newark, Essex County,..."
1,1155-1157 MCCARTER HWY,LEG 450 BROAD STREET LLC,283-299 MARKET ST,"NEWARK, NJ 07102",1,"[1155-1157 MCCARTER HWY, 2-28 DIVISION ST, 30-...",4,"1155-1157 MCCARTER HWY Newark, NJ","(McCarter Highway, Newark, Essex County, New J..."
2,23 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"23 BRIDGE ST Newark, NJ","(23, Bridge Street, Newark, Essex County, New ..."
3,25 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NY 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"25 BRIDGE ST Newark, NJ","(25, Bridge Street, Newark, Essex County, New ..."
4,27 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"27 BRIDGE ST Newark, NJ","(27, Bridge Street, Newark, Essex County, New ..."
...,...,...,...,...,...,...,...,...,...
6888,130 CHESTER AVE,"SANCHEZ, FRANCISCO ROSA",130 CHESTER AVE,"NEWARK, NJ 07104",1,[130 CHESTER AVE],1,"130 CHESTER AVE Newark, NJ","(130, Chester Avenue, Newark, Essex County, Ne..."
7619,55 CHESTER AVE,"TORRES, RUBEN JR",55 CHESTER AVE,"NEWARK, NJ 07104",1,[55 CHESTER AVE],1,"55 CHESTER AVE Newark, NJ","(55, Chester Avenue, Newark, Essex County, New..."
7714,119 CHESTER AVE,"PRIETO, SERGIO M.",119 CHESTER AVE,"NEWARK, NJ 07104",1,[119 CHESTER AVE],1,"119 CHESTER AVE Newark, NJ","(119, Chester Avenue, Newark, Essex County, Ne..."
34606,309 AVENUE P,"PORT HOLDINGS, LLC",572 MARKET ST,"NEWARK, NJ 07105",1,"[309-327 AVENUE P, 309-327 AVENUE P REAR, 339-...",3,"309 AVENUE P Newark, NJ","(309, Avenue P, Newark, Essex County, New Jers..."


In [225]:
errors_df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
2455,594 S 13TH,CORINTHIAN HOMES II URB RNWL,595 S 11TH ST,"NEWARK, NJ 07103",1,"[595-607 S 12TH ST, 594-602 S 13TH, 625-641 S ...",3,"594 S 13TH Newark, NJ",None
2957,520 S 16TH S,"ASCENSION CAPITAL PARTNERS I,LLC","59 LINCOLN PARK,#200","NEWARK, NJ 07102",1,[520-526 S 16TH S],1,"520 S 16TH S Newark, NJ",None
3461,766 S 20TH,"FAIRFAX, DIANE",304 LIVINGSTON ST,"WESTFIELD, NJ 07090",1,[766 S 20TH],1,"766 S 20TH Newark, NJ",None
3688,81 C0SSIO DR,"JACKSON, DWAYNE & ANNA",81 COSSIO DR,"NEWARK, NJ 07103",1,[81 C0SSIO DR],1,"81 C0SSIO DR Newark, NJ",None
7217,532 N 13TH,"BAPTISTA, FILIPE A.",532 N 13TH ST,"NEWARK, NJ 07107",1,[532-534 N 13TH],1,"532 N 13TH Newark, NJ",None
...,...,...,...,...,...,...,...,...,...
34633,121 AVENUE I REAR,"TERRENO STOCKTON LLC,",101 MONTGOMERY ST #200,"SAN FRANCISCO, CA 94104",1,"[121-137 AVENUE I REAR, 121 AVENUE I REAR]",2,"121 AVENUE I REAR Newark, NJ",None
34683,386 RT 1 & 9,"CFS 2907 NEWARK SKYP,C/O RYAN",600 5TH AVE. 9TH FL,"NEW YORK, NY 10020",1,"[386-406 RT 1 & 9, 362-384 RT 1 & 9]",2,"386 RT 1 & 9 Newark, NJ",None
34684,362 RT 1 & 9,"CFS 2907 NEWARK SKYP,C/O RYAN",600 5TH AVE 9TH FL,"NEW YORK, NY 10020",1,"[386-406 RT 1 & 9, 362-384 RT 1 & 9]",2,"362 RT 1 & 9 Newark, NJ",None
34685,106 RT 1 & 9,CFS2907 NWK MCLELLAN LLC C/O RYAN,600 5TH AVE. 9TH FL,"NEW YORK, NY 10020",1,[106-132 RT 1 & 9],1,"106 RT 1 & 9 Newark, NJ",None


In [226]:
errors_df.propertyLocation.unique()

array(['594 S 13TH', '520 S 16TH S', '766 S 20TH', '81 C0SSIO DR',
       '532 N 13TH', '214 BALLANTINE PKW', '252 W00DSIDE AVE',
       '100 FOREST HILL PK', '71 FOREST HILL CONDO',
       '77 FOREST HILL CONDO', '47 BRANCH BROOK NY', '72 COLOMBIA ST',
       '233 NO 4TH ST', '96 CLIFORD ST', 'VARNUM ST', '27 WHEELR PT RD',
       '205 VANDERPOL ST', '557 SOUTH ORNAGE AVE', '112 ELEVENTH AVE',
       '241 18TH ST', '110 S 15TH S', 'BLDG551 ORANGE ST', '103 N 13TH',
       '355 ORTH 12TH ST', '243 2ND AVEEST', '17 PATERSON ST H1',
       '86 LISTER AVE  REAR', '10 HORATIOST', '8 DOLBY PL',
       '752 S FIFTEETH ST', '764 SO 18TH', '843 S 19TH', '813 S 18TH',
       '813 S 17TH', '813 S 14TH', '813 S 13TH', '798 S 14TH',
       '816 S 13TH', '510 IRVINE TURNER BL', '514 IRVINE TURNER BL',
       '297- M L KING BLVD', '65 NORFLOK ST', '823 S 13TH', '843 S 14TH',
       '881 S 14TH', '908 S 16TH', '888 S 16TH', '881 S 16TH',
       '923 S 16TH', '881 S 17TH', '893 S 17TH', '878 S 18TH',


In [229]:
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MCWHORTER', 'MC WHORTER')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ML KING', 'DR MARTIN LUTHER KING')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('DR MARTIN LUTHER KING', 'DR MARTIN LUTHER KING JR')

# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SCOFIELD', 'SCHOFIELD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('STECHER', 'STETCHER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PARK VIEW', 'PARKVIEW')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SANDFORD', 'SANFORD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VAL SUMO', 'VALSUMO')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PATTERSON', 'PATERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE E', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('13COLGATE', '13 COLGATE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MOUNTAIN VIEW', 'MOUNTAINVIEW')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('DE GRAW', 'DEGRAW')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GOTTHART', 'GOTTHARDT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE W', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ST AR', 'ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVD AR', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE AR', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NJRR', 'New Jersey Railroad')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HWY AR', 'HWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PL AR', 'PL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LN AR', 'LN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HOMESTEAD PK RD', 'HOMESTEAD PARK RD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('13TH R', '13TH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HWY R', 'HWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE R', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ST R', 'ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE,', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE  AR', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ELVENTH', 'ELEVENTH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THRITEENTH R', '13TH ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PKWAY', 'PKWY')

#errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{1,4}', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('STEAR', 'ST')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVEEAR', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVDREAR', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVD R', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE  E', 'AVE')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE ,E', 'AVE')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HALSTED', 'HALSTEAD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVDEAR', 'BLVD')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HWYEAR', 'HWY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(', REAR', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PL REAR', 'PL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MNCHESTER', 'MANCHESTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MANCHESTRER', 'MANCHESTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('STUYVSNT', 'STUYVESANT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVEN10TH', '17TH')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVNTEENTH', '17TH')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BLVDE', 'BLVD')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ELLIOT', 'ELLIOTT')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HENNESY', 'HENNESSEY')
#errors_df.propertyLocation = errors_df.propertyLocation.str.replace('P REAR', 'P')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOREST', 'FORREST')
errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\\d{0,2}TH', r'\\d{0,2}TH ST')


errors_df['propertyFullAddress'] = errors_df.propertyLocation + ' Newark, NJ'

/var/folders/jn/0t9fgm4d05bdfp70phhgc1sr0000gn/T/ipykernel_1997/564474028.py:57: FutureWarning: The default value of regex will change from True to False in a future version.
  errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\\d{0,2}TH', r'\\d{0,2}TH ST')


In [230]:
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [231]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [232]:
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
2455,594 S 13TH,CORINTHIAN HOMES II URB RNWL,595 S 11TH ST,"NEWARK, NJ 07103",1,"[595-607 S 12TH ST, 594-602 S 13TH, 625-641 S ...",3,"594 S 13TH Newark, NJ",None
2957,520 S 16TH S,"ASCENSION CAPITAL PARTNERS I,LLC","59 LINCOLN PARK,#200","NEWARK, NJ 07102",1,[520-526 S 16TH S],1,"520 S 16TH S Newark, NJ",None
3461,766 S 20TH,"FAIRFAX, DIANE",304 LIVINGSTON ST,"WESTFIELD, NJ 07090",1,[766 S 20TH],1,"766 S 20TH Newark, NJ",None
3688,81 C0SSIO DR,"JACKSON, DWAYNE & ANNA",81 COSSIO DR,"NEWARK, NJ 07103",1,[81 C0SSIO DR],1,"81 C0SSIO DR Newark, NJ",None
7217,532 N 13TH,"BAPTISTA, FILIPE A.",532 N 13TH ST,"NEWARK, NJ 07107",1,[532-534 N 13TH],1,"532 N 13TH Newark, NJ",None
...,...,...,...,...,...,...,...,...,...
34633,121 AVENUE I REAR,"TERRENO STOCKTON LLC,",101 MONTGOMERY ST #200,"SAN FRANCISCO, CA 94104",1,"[121-137 AVENUE I REAR, 121 AVENUE I REAR]",2,"121 AVENUE I REAR Newark, NJ",None
34683,386 RT 1 & 9,"CFS 2907 NEWARK SKYP,C/O RYAN",600 5TH AVE. 9TH FL,"NEW YORK, NY 10020",1,"[386-406 RT 1 & 9, 362-384 RT 1 & 9]",2,"386 RT 1 & 9 Newark, NJ",None
34684,362 RT 1 & 9,"CFS 2907 NEWARK SKYP,C/O RYAN",600 5TH AVE 9TH FL,"NEW YORK, NY 10020",1,"[386-406 RT 1 & 9, 362-384 RT 1 & 9]",2,"362 RT 1 & 9 Newark, NJ",None
34685,106 RT 1 & 9,CFS2907 NWK MCLELLAN LLC C/O RYAN,600 5TH AVE. 9TH FL,"NEW YORK, NY 10020",1,[106-132 RT 1 & 9],1,"106 RT 1 & 9 Newark, NJ",None


In [222]:
#non NA values appened to the dataframe and then pickled
df.append(errors_df.dropna(axis=0, subset=['gCode'])).to_pickle('newark.pkl')
# writing over the old errors_df
errors_df = errors_df.iloc[geocode_errors].copy()
# saving the new errors_df
errors_df.to_pickle('errors_newark.pkl')

In [79]:
first_cleaning = timedelta(seconds=17439)
second_cleaning = timedelta(seconds=819.5)

In [95]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]
errors_df.iloc[geocode_errors]

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode


In [86]:
errors_df.iloc[[i for i,e in enumerate(errors_df.gCode) if e != None]] == errors_df.dropna(axis=0, subset=['gCode'])

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
522,True,True,True,True,True,True,True,True,True
523,True,True,True,True,True,True,True,True,True
524,True,True,True,True,True,True,True,True,True
525,True,True,True,True,True,True,True,True,True
730,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
34501,True,True,True,True,True,True,True,True,True
34502,True,True,True,True,True,True,True,True,True
34503,True,True,True,True,True,True,True,True,True
34504,True,True,True,True,True,True,True,True,True


In [100]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
0,1050-1052 MCCARTER HWY,SEABRA BROTHERS II,260CHESTNUT ST. 2ND FLOOR,"NEWARK, NJ 07105",1,[1050-1052 MCCARTER HWY],1,"1050-1052 MCCARTER HWY Newark, NJ","(1050, McCarter Highway, Newark, Essex County,..."
1,1155-1157 MCCARTER HWY,LEG 450 BROAD STREET LLC,283-299 MARKET ST,"NEWARK, NJ 07102",1,"[1155-1157 MCCARTER HWY, 2-28 DIVISION ST, 30-...",4,"1155-1157 MCCARTER HWY Newark, NJ","(McCarter Highway, Newark, Essex County, New J..."
2,23 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"23 BRIDGE ST Newark, NJ","(23, Bridge Street, Newark, Essex County, New ..."
3,25 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NY 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"25 BRIDGE ST Newark, NJ","(25, Bridge Street, Newark, Essex County, New ..."
4,27 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"27 BRIDGE ST Newark, NJ","(27, Bridge Street, Newark, Essex County, New ..."
...,...,...,...,...,...,...,...,...,...
34710,21-23 GARIBALDI AVE,"PAULO,M. JIMMY & SUSAN",445 KING GEORGE RD,"BASKING RIDGE, NJ 07920",1,[21-23 GARIBALDI AVE],1,"21-23 GARIBALDI AVE Newark, NJ","(21, Garibaldi Avenue, Newark, Essex County, N..."
34711,25-27 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"25-27 GARIBALDI AVE Newark, NJ","(25, Garibaldi Avenue, Newark, Essex County, N..."
34712,29 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"29 GARIBALDI AVE Newark, NJ","(29, Garibaldi Avenue, Newark, Essex County, N..."
34713,31 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"31 GARIBALDI AVE Newark, NJ","(31, Garibaldi Avenue, Newark, Essex County, N..."


In [87]:
df.append(errors_df.dropna(axis=0, subset=['gCode'], inplace=True))

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
0,1050-1052 MCCARTER HWY,SEABRA BROTHERS II,260CHESTNUT ST. 2ND FLOOR,"NEWARK, NJ 07105",1,[1050-1052 MCCARTER HWY],1,"1050-1052 MCCARTER HWY Newark, NJ","(1050, McCarter Highway, Newark, Essex County,..."
1,1155-1157 MCCARTER HWY,LEG 450 BROAD STREET LLC,283-299 MARKET ST,"NEWARK, NJ 07102",1,"[1155-1157 MCCARTER HWY, 2-28 DIVISION ST, 30-...",4,"1155-1157 MCCARTER HWY Newark, NJ","(McCarter Highway, Newark, Essex County, New J..."
2,23 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"23 BRIDGE ST Newark, NJ","(23, Bridge Street, Newark, Essex County, New ..."
3,25 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NY 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"25 BRIDGE ST Newark, NJ","(25, Bridge Street, Newark, Essex County, New ..."
4,27 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"27 BRIDGE ST Newark, NJ","(27, Bridge Street, Newark, Essex County, New ..."
...,...,...,...,...,...,...,...,...,...
34710,21-23 GARIBALDI AVE,"PAULO,M. JIMMY & SUSAN",445 KING GEORGE RD,"BASKING RIDGE, NJ 07920",1,[21-23 GARIBALDI AVE],1,"21-23 GARIBALDI AVE Newark, NJ","(21, Garibaldi Avenue, Newark, Essex County, N..."
34711,25-27 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"25-27 GARIBALDI AVE Newark, NJ","(25, Garibaldi Avenue, Newark, Essex County, N..."
34712,29 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"29 GARIBALDI AVE Newark, NJ","(29, Garibaldi Avenue, Newark, Essex County, N..."
34713,31 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"31 GARIBALDI AVE Newark, NJ","(31, Garibaldi Avenue, Newark, Essex County, N..."


In [88]:
df.to_pickle('newark.pkl')

In [89]:
df = pd.read_pickle('newark.pkl')

In [90]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
0,1050-1052 MCCARTER HWY,SEABRA BROTHERS II,260CHESTNUT ST. 2ND FLOOR,"NEWARK, NJ 07105",1,[1050-1052 MCCARTER HWY],1,"1050-1052 MCCARTER HWY Newark, NJ","(1050, McCarter Highway, Newark, Essex County,..."
1,1155-1157 MCCARTER HWY,LEG 450 BROAD STREET LLC,283-299 MARKET ST,"NEWARK, NJ 07102",1,"[1155-1157 MCCARTER HWY, 2-28 DIVISION ST, 30-...",4,"1155-1157 MCCARTER HWY Newark, NJ","(McCarter Highway, Newark, Essex County, New J..."
2,23 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"23 BRIDGE ST Newark, NJ","(23, Bridge Street, Newark, Essex County, New ..."
3,25 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NY 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"25 BRIDGE ST Newark, NJ","(25, Bridge Street, Newark, Essex County, New ..."
4,27 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"27 BRIDGE ST Newark, NJ","(27, Bridge Street, Newark, Essex County, New ..."
...,...,...,...,...,...,...,...,...,...
34710,21-23 GARIBALDI AVE,"PAULO,M. JIMMY & SUSAN",445 KING GEORGE RD,"BASKING RIDGE, NJ 07920",1,[21-23 GARIBALDI AVE],1,"21-23 GARIBALDI AVE Newark, NJ","(21, Garibaldi Avenue, Newark, Essex County, N..."
34711,25-27 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"25-27 GARIBALDI AVE Newark, NJ","(25, Garibaldi Avenue, Newark, Essex County, N..."
34712,29 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"29 GARIBALDI AVE Newark, NJ","(29, Garibaldi Avenue, Newark, Essex County, N..."
34713,31 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"31 GARIBALDI AVE Newark, NJ","(31, Garibaldi Avenue, Newark, Essex County, N..."


In [78]:
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [ ]:
df.dropna(axis=0, subset=['gCode'], inplace=True)

In [ ]:
df

,propertyLocation,ownersName,ownersMailingAddress,cityStateZip,units,propertiesOwned,numberPropertiesOwned,propertyFullAddress,gCode
0,1050-1052 MCCARTER HWY,SEABRA BROTHERS II,260CHESTNUT ST. 2ND FLOOR,"NEWARK, NJ 07105",1,[1050-1052 MCCARTER HWY],1,"1050-1052 MCCARTER HWY Newark, NJ","(1050, McCarter Highway, Newark, Essex County,..."
1,1155-1157 MCCARTER HWY,LEG 450 BROAD STREET LLC,283-299 MARKET ST,"NEWARK, NJ 07102",1,"[1155-1157 MCCARTER HWY, 2-28 DIVISION ST, 30-...",4,"1155-1157 MCCARTER HWY Newark, NJ","(McCarter Highway, Newark, Essex County, New J..."
2,23 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"23 BRIDGE ST Newark, NJ","(23, Bridge Street, Newark, Essex County, New ..."
3,25 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NY 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"25 BRIDGE ST Newark, NJ","(25, Bridge Street, Newark, Essex County, New ..."
4,27 BRIDGE ST,"ALS NEWARK LLC,",494 BROAD ST SUITE 210,"NEWARK, NJ 07102",1,"[23 BRIDGE ST, 25 BRIDGE ST, 27 BRIDGE ST]",3,"27 BRIDGE ST Newark, NJ","(27, Bridge Street, Newark, Essex County, New ..."
...,...,...,...,...,...,...,...,...,...
34710,21-23 GARIBALDI AVE,"PAULO,M. JIMMY & SUSAN",445 KING GEORGE RD,"BASKING RIDGE, NJ 07920",1,[21-23 GARIBALDI AVE],1,"21-23 GARIBALDI AVE Newark, NJ","(21, Garibaldi Avenue, Newark, Essex County, N..."
34711,25-27 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"25-27 GARIBALDI AVE Newark, NJ","(25, Garibaldi Avenue, Newark, Essex County, N..."
34712,29 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"29 GARIBALDI AVE Newark, NJ","(29, Garibaldi Avenue, Newark, Essex County, N..."
34713,31 GARIBALDI AVE,"LUSA ENTERPRISE, INC",52-54 PORETE AVE,"NORTH ARLINGTON, NJ 07031",1,"[39-41 PERSHING AVE, 43-53 PERSHING AVE, 28-32...",9,"31 GARIBALDI AVE Newark, NJ","(31, Garibaldi Avenue, Newark, Essex County, N..."


In [ ]:
errors_df.to_pickle('errors_newark.pkl')

In [ ]:
#errors_df['propertyFullAddress'] = errors_df.propertyLocation + ' Newark, NJ'

In [ ]:
df.append(errors_df.dropna(axis=0, subset=['gCode']))

In [ ]:
errors_df = errors_df.iloc[geocode_errors].copy()

In [ ]:
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [ ]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [ ]:
errors_df.iloc[geocode_errors]['propertyLocation']

2455          594 S 13TH
2957        520 S 16TH S
3102     587 S SEVEN10TH
3461          766 S 20TH
3503         5 GREEK WAY
              ...       
34662      857 DOMUS AVE
34683       386 RT 1 & 9
34684       362 RT 1 & 9
34685       106 RT 1 & 9
34686       314 RT 1 & 9
Name: propertyLocation, Length: 183, dtype: object

In [ ]:
errors_df.to_pickle('errors_df.pkl')